In [1]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

In [3]:
%%sql
drop table if exists tbl;
create table tbl (
    name varchar(32) primary key,
    point integer not null check (point >= 0 and point <= 100)
);

insert into tbl
values ('Alice', 68),
    ('Bob', 79),
    ('Carol', 81),
    ('Dave', 0),
    ('Eve', 100);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
5 rows affected.


[]

## やりたいこと
+ 点数によってA~Dに分類する(Rの`cut`相当)
    + A: 91~100
    + B: 81~90
    + C: 71~80
    + D: それ以下

In [5]:
%%sql
-- 素直にcase文で
select *,
    case
        when point between 0 and 70 then 'D'
        when point between 71 and 80 then 'C'
        when point between 81 and 90 then 'B'
        when point between 91 and 100 then 'A'
        else null
    end as grade
from tbl

*  postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


name,point,grade
Alice,68,D
Bob,79,C
Carol,81,B
Dave,0,D
Eve,100,A


In [8]:
%%sql
-- updateする場合もcase文が有効
ALTER TABLE tbl
ADD if not exists grade char(1);

update tbl
set grade = case
        when point between 0 and 70 then 'D'
        when point between 71 and 80 then 'C'
        when point between 81 and 90 then 'B'
        when point between 91 and 100 then 'A'
        else null
    end;

select *
from tbl;

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
5 rows affected.
5 rows affected.


name,point,grade
Alice,68,D
Bob,79,C
Carol,81,B
Dave,0,D
Eve,100,A


In [7]:
%%sql
create table Grades (
    low integer not null,
    high integer not null,
    label char(1) check (label in ('A', 'B', 'C', 'D')),
    primary key(low, high)
);
insert into Grades
values (91,100, 'A'),
    (81,90, 'B'),
    (71,80, 'C'),
    (0,70,'D');

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
4 rows affected.


[]

In [8]:
%%sql
-- Gradesというテーブルを作ってjoinする
select t.*, g.label
from tbl as t
inner join Grades as g
on t.point between g.low and g.high

*  postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


name,point,label
Alice,68,D
Bob,79,C
Carol,81,B
Dave,0,D
Eve,100,A
